In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# **Dataset**
1. Jester Dataset 1 contains 24983 rows and 101 columns.
2. Each row corresponds to a user.
3. First column is the number of jokes rated by that user.
4. The numbers refer to the ID's of the jokes in the Excel files.
5. Others are ratings for each joke. Ratings are in the range of -10 to 10. Value of 99 indicates Null, which is no rating.


In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/zeitgeist-hash/CE888-1/master/lab4-recommender/jester-data-1.csv", header=None)
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,74,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,-4.76,-8.50,-6.75,-7.18,8.45,-7.18,-7.52,-7.43,-9.81,-9.85,-9.85,-9.37,1.50,-4.37,-9.81,-8.50,1.12,7.82,2.86,9.13,-7.43,2.14,-4.08,-9.08,7.82,5.05,4.95,-9.17,-8.40,-8.40,...,8.59,3.59,-6.84,-9.03,2.82,-1.36,-9.08,8.30,5.68,-4.81,99.00,99.00,99.00,99.00,99.00,99.00,99.00,-9.42,99.00,99.00,99.00,-7.72,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,2.82,99.00,99.00,99.00,99.00,99.00,-5.63,99.00,99.00,99.00
1,100,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,6.75,8.64,4.42,7.43,4.56,-0.97,4.66,-0.68,3.30,-1.21,0.87,8.64,8.35,9.17,0.05,7.57,4.71,0.87,-0.39,6.99,6.50,-0.92,7.14,9.03,-1.80,0.73,7.09,3.40,-0.87,...,-6.70,-3.35,-9.03,4.47,4.08,-3.83,8.74,1.12,0.78,7.52,-5.00,2.77,8.30,7.77,7.33,6.21,7.72,8.98,8.64,8.20,3.93,4.85,4.85,6.07,8.98,4.51,-0.05,3.69,4.56,0.58,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,49,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,99.00,99.00,7.33,7.57,9.37,6.17,-6.36,-6.89,-7.86,9.03,9.03,9.03,7.28,99.00,8.25,99.00,99.00,7.48,7.28,7.28,8.93,99.00,6.17,7.28,99.00,99.00,8.98,7.33,99.00,6.17,9.08,...,6.46,7.28,99.00,99.00,7.04,7.28,99.00,7.28,8.25,99.00,99.00,99.00,99.00,99.00,99.00,8.93,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,9.03,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00
3,48,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,99.00,1.84,7.33,6.60,6.31,8.11,-7.23,-6.65,1.17,-6.60,-3.64,-2.09,5.34,99.00,99.00,99.00,99.00,2.91,3.93,6.75,6.60,99.00,6.65,-6.12,99.00,7.57,6.21,6.65,99.00,-8.30,7.18,...,0.00,-3.69,99.00,99.00,7.82,0.24,99.00,7.28,-2.33,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,0.63,99.00,99.00,-2.33,99.00,99.00,99.00,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00
4,91,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,8.25,6.84,-3.93,7.23,-2.33,-9.66,2.72,-1.36,2.57,4.51,8.20,6.12,8.30,-1.26,7.77,1.89,-1.17,5.68,8.45,4.61,8.06,-9.47,7.28,5.68,2.48,3.20,-1.26,6.80,4.51,...,7.38,6.17,4.71,-2.28,7.38,4.56,7.14,4.22,3.01,3.83,99.00,99.00,99.00,99.00,99.00,99.00,4.13,99.00,99.00,99.00,5.24,5.92,0.87,7.28,3.93,-0.63,6.31,4.71,2.82,2.96,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24978,100,0.44,7.43,9.08,2.33,3.20,6.75,-8.79,-0.53,-8.74,7.23,-0.53,5.63,-7.14,-4.08,-3.50,-8.20,-3.98,-9.22,-0.15,-6.46,5.63,-0.92,-2.91,-4.17,2.82,3.40,8.64,6.84,6.80,-0.87,7.38,-3.50,8.88,7.43,5.39,2.23,-0.68,3.40,-0.58,...,8.59,3.45,0.87,9.27,-4.66,5.73,-0.49,8.35,1.94,5.00,-9.66,8.98,8.98,-9.81,9.13,9.08,9.08,3.98,0.73,9.03,8.98,9.22,8.93,9.13,9.27,-1.99,-9.95,-9.90,9.13,8.83,8.83,-1.21,9.22,-6.70,8.45,9.03,6.55,8.69,8.79,7.43
24979,91,9.13,-8.16,8.59,9.08,0.87,-8.93,-3.50,5.78,-8.11,4.90,8.88,-8.69,-7.48,-8.83,-1.75,6.60,3.54,1.50,7.67,-0.44,9.22,8.74,9.03,9.08,8.93,3.74,3.20,-9.17,-8.98,8.79,-7.67,-3.06,9.13,8.40,-0.63,-7.18,0.58,8.88,9.27,...,2.77,8.11,-7.96,8.93,-0.87,-5.87,8.88,-1.12,-8.74,8.74,99.00,99.00,99.00,99.00,99.00,4.90,99.00,99.00,99.00,99.00,-0.29,0.92,-0.78,0.15,-0.10,0.00,-0.19,-0.87,-1.36,-0.58,-1.17,-5.73,-1.46,0.24,9.22,-8.20,-7.23,-8.59,9.13,8.45
24980,39,99.00,99.00,99.00,99.00,-7.77,99.00,6.70,-6.75,99.00,99.00,99.00,99.00,-6.46,-1.65,-6.80,-6.41,-6.99,7.23,6.75,-6.99,6.55,99.00,99.00,99.00,99.00,0.49,-0.53,-6.94,-0.49,99.00,6.46,-0.53,99.00,99.00,-7.86,-0.34,99.00,-6.94,99.00,...,0.49,-0.24,99.00,99.00,-3.11,-6.65,99.00,-0.58,6.31,99.00,99.00,-7.86,99.00,99.00,

In [ ]:
df = df.loc[:, 1:]
arr = df.values
print(arr.shape)

(24983, 100)


In [ ]:
rated = np.where(arr!=99)
print(len(rated[0]), rated[1].shape)

1810455 (1810455,)


In [ ]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [ ]:
new_arr, idx = replace(arr, 0.1)

In [ ]:
arr[idx[0][0], idx[1][0]]

-6.07

In [ ]:
new_arr[idx[0][0], idx[1][0]]

99.0

# **Latent Factor**

In [ ]:
#Set latent factors as 50
n_latent_factors = 50

user_ratings = new_arr

# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

In [ ]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001, weight_decay=0.1):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * (latent_item_features[item_id])
    latent_item_features[item_id] -= alpha * err * (user_pref_values)
    return err
    

def sgd(iterations):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if (rating!=99):
                # if not np.isnan(rating):
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        # if (iteration % 100) == 0:
        #     print('MSE ', iteration, ': ', mse)
        print('MSE ', iteration, ': ', mse)

In [ ]:
num_iter = 50
sgd(num_iter)

MSE  0 :  28.638563547765674
MSE  1 :  25.036577403845012
MSE  2 :  24.950843168405733
MSE  3 :  24.88232813862681
MSE  4 :  24.822793425183622
MSE  5 :  24.765265482877627
MSE  6 :  24.702795996820964
MSE  7 :  24.627318347994162
MSE  8 :  24.528461721204284
MSE  9 :  24.392325779340705
MSE  10 :  24.20048317843752
MSE  11 :  23.929975788875364
MSE  12 :  23.55580667700697
MSE  13 :  23.05791758616145
MSE  14 :  22.433372014292814
MSE  15 :  21.709528495516462
MSE  16 :  20.947186128613307
MSE  17 :  20.223739555739016
MSE  18 :  19.602706013196748
MSE  19 :  19.112016727293334
MSE  20 :  18.745335107225046
MSE  21 :  18.478260531883635
MSE  22 :  18.283609465275646
MSE  23 :  18.13889671858224
MSE  24 :  18.0277760161413
MSE  25 :  17.938983564781378
MSE  26 :  17.86486609593892
MSE  27 :  17.80019402804844
MSE  28 :  17.74134243893592
MSE  29 :  17.685755342889532
MSE  30 :  17.63160041546766
MSE  31 :  17.577546704181152
MSE  32 :  17.522621287937707
MSE  33 :  17.466116798607366
M

In [ ]:
def validation_check():
  error = []
  prediction = latent_user_preferences.dot(latent_item_features.T)
  for i in range(len(idx[0])):
    predicted_rating = prediction[idx[0][i]][idx[1][i]]
    actual_rating = arr[idx[0][i]][idx[1][i]]
    err = predicted_rating - actual_rating
    error.append(err)
  mse = (np.array(error) ** 2).mean()
  print('Validation MSE: ', mse)

In [ ]:
validation_check()

Validation MSE:  17.815538549236273


In [ ]:
# Creating new validation arrays from original data randomly again
for i in range(10):
  new_arr, idx = replace(arr, 0.1)
  validation_check()

Validation MSE:  16.284825032090804
Validation MSE:  16.43737996935927
Validation MSE:  16.31993557088579
Validation MSE:  16.431960569166325
Validation MSE:  16.46936076758466
Validation MSE:  16.341887194289257
Validation MSE:  16.405127686515147
Validation MSE:  16.41729826580208
Validation MSE:  16.32430044037324
Validation MSE:  16.50220766751229


# Results






In [ ]:
prediction = latent_user_preferences.dot(latent_item_features.T)

In [ ]:
# Prediction values on entire Test Dataset

values = [zip(df.values[i], prediction[i]) for i in range(prediction.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = df.columns

# For each data point, the number on the left is the original value from the dataset, the number on the right is the prediction
comparison_data

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,"(-7.82, -3.7179174396851313)","(8.79, -1.6132444596295863)","(-9.66, -6.587026600558394)","(-8.16, -8.864964180605547)","(-7.52, -2.7323146101562927)","(-8.5, -2.708623468785011)","(-9.85, -3.5951564786541055)","(4.17, -4.003113626824221)","(-8.98, -7.9693652067344845)","(-4.76, -4.919980034299686)","(-8.5, -3.5761238129057493)","(-6.75, -2.505636192703524)","(-7.18, -3.9944529755579827)","(8.45, -1.430567115012328)","(-7.18, -3.9616852433515035)","(-7.52, -6.891802814448163)","(-7.43, -3.5222614611658978)","(-9.81, -3.625597106918512)","(-9.85, -3.4335150883778143)","(-9.85, -5.577780337592808)","(-9.37, -0.47756755482668883)","(1.5, -3.9585494514359847)","(-4.37, -5.757736163807887)","(-9.81, -9.506376528231172)","(-8.5, -3.1847762254778873)","(1.12, -1.4921313302649524)","(7.82, 0.9427077352438069)","(2.86, -3.0282886100508106)","(9.13, 2.0361479362418704)","(-7.43, -7.977285791390199)","(2.14, -0.29520733741846666)","(-4.08, 1.0026230478286706)","(-9.08, -8.402045804084873)","(7.82, -2.548109593361306)","(5.05, 0.7680408561992815)","(4.95, 1.2587310625607029)","(-9.17, -7.277387738399995)","(-8.4, -4.201445708135123)","(-8.4, -3.3101359207549423)","(-8.4, -3.644299161337537)",...,"(8.59, -1.0423302340920166)","(3.59, -0.38571266538986226)","(-6.84, -5.240647060688812)","(-9.03, -7.476561846529644)","(2.82, 0.17531410812477904)","(-1.36, -0.22208032985391793)","(-9.08, -7.4287706183454585)","(8.3, -0.38927128908098685)","(5.68, 0.6217134850804468)","(-4.81, -4.585389904387626)","(99.0, -5.413351852796263)","(99.0, -2.102184901267556)","(99.0, -6.200179563616615)","(99.0, -8.297481201322736)","(99.0, -8.41360065112378)","(99.0, -2.2883821210874173)","(99.0, -6.517700076249997)","(-9.42, -5.74819351077247)","(99.0, -7.209475504269819)","(99.0, -5.075133796094891)","(99.0, -2.5799732122615624)","(-7.72, -5.357751559563038)","(99.0, -3.1204707988460543)","(99.0, -5.784226368622381)","(99.0, -6.0883871532247955)","(99.0, -6.171643276871328)","(99.0, -3.2991986810649223)","(99.0, -3.093584699665226)","(99.0, 0.1523183449095052)","(99.0, -6.354573237915222)","(2.82, -2.6709180172589866)","(99.0, -4.512322109622376)","(99.0, -2.597526526706805)","(99.0, -5.581896127694096)","(99.0, -5.4518429433056745)","(99.0, -4.196133425529526)","(-5.63, -3.5358045400962466)","(99.0, -3.868456786358891)","(99.0, -6.636807414403714)","(99.0, -4.79978010556495)"
1,"(4.08, 3.778234904017037)","(-0.29, 0.8569751976491499)","(6.36, 4.2417603986677594)","(4.37, 4.206918167579914)","(-2.38, 0.15439316659480531)","(-9.66, 2.0613067926144932)","(-0.73, 1.219695202804141)","(-5.34, 1.3272211377288876)","(8.88, 5.628604394518986)","(9.22, 5.184899420810397)","(6.75, 2.7291886466158592)","(8.64, 1.8343534414535725)","(4.42, -0.04688635076108584)","(7.43, 1.1722165273652387)","(4.56, -0.13408784138851446)","(-0.97, 1.6152784984881383)","(4.66, -0.1987955143020759)","(-0.68, 0.973138558203871)","(3.3, 1.8778456855436636)","(-1.21, 1.1860046306631322)","(0.87, 1.0410708005956222)","(8.64, 3.2486154643208485)","(8.35, 4.5955160502113594)","(9.17, 6.136983372521145)","(0.05, 2.087628108610292)","(7.57, 0.7422271202531185)","(4.71, 1.3903638602077044)","(0.87, 1.8651699123988092)","(-0.39, 0.3130115744052067)","(6.99, 3.997240199211408)","(6.5, 0.8831413622525389)","(-0.92, 1.5573920068899925)","(7.14, 4.045651937973875)","(9.03, 2.4149916138259466)","(-1.8, 1.4728550336431858)","(0.73, 1.310749716936902)","(7.09, 2.558462547892566)","(3.4, 2.965931236303718)","(-0.87, 1.8569361367958228)","(7.91, 2.0301302746312286)",...,"(-6.7, 0.3411849434479511)","(-3.35, 1.5397992157950304)","(-9.03, 2.3155030545752338)","(4.47, 4.951082678328272)","(4.08, 0.7164417073403598)","(-3.83, 1.2004208815437338)","(8.74, 6.502119192635598)","(1.1

In [ ]:
jokes_ratings = pd.DataFrame(prediction)
mean_idx = jokes_ratings.mean()
min_idx = mean_idx.idxmin()
max_idx = mean_idx.idxmax()
print('Worst Joke ID predicted: ', min_idx, '; Average Rating: ', mean_idx[min_idx])
print('Best Joke ID predicted: ', max_idx, '; Average Rating: ', mean_idx[max_idx])

Worst Joke ID predicted:  57 ; Average Rating:  -4.111693292049609
Best Joke ID predicted:  49 ; Average Rating:  3.521651851012798
